In [1]:
using CSV, DataFrames, Gadfly, GLM, Statistics, LinearAlgebra, Distributions, Combinatorics, StatsBase, MLBase, Random

In [145]:
include("functions.jl");

In [146]:
data = CSV.read("train.csv", DataFrame)

first(data,5)

,ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG
,Int64,Int64,String,String,Int64,Int64?,Int64,String
1,1,69,M,ASY,140,110,1,Normal
2,2,60,M,ASY,140,293,0,LVH
3,3,52,M,ASY,165,missing,1,Normal
4,4,46,M,NAP,120,230,0,Normal
5,5,61,M,NAP,120,missing,0,Normal


In [147]:
countmap(data.HeartDisease)

Dict{Int64,Int64} with 2 entries:
  0 => 315
  1 => 373

In [148]:
missing_training = DataFrame()
train_cholesterol = DataFrame()


for row in eachrow(data)
    # print(ismissing(row.Cholesterol) == true, '\n')
    if ismissing(row.Cholesterol)
        push!(missing_training, row)
    else
        push!(train_cholesterol, row)
    end
end



In [149]:
first(missing_training, 5)


,ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG
,Int64,Int64,String,String,Int64,Missing,Int64,String
1,3,52,M,ASY,165,missing,1,Normal
2,5,61,M,NAP,120,missing,0,Normal
3,18,57,M,ASY,95,missing,1,Normal
4,31,38,M,NAP,100,missing,0,Normal
5,43,56,M,NAP,125,missing,1,Normal


In [150]:
first(train_cholesterol, 5)

,ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG
,Int64,Int64,String,String,Int64,Int64,Int64,String
1,1,69,M,ASY,140,110,1,Normal
2,2,60,M,ASY,140,293,0,LVH
3,4,46,M,NAP,120,230,0,Normal
4,6,46,M,NAP,150,231,0,Normal
5,7,45,F,ATA,112,160,0,Normal


In [ ]:
# function GetEncodedFormRow(dataframe:DataFrame)

# end

# function GetEncodedFormDataframe(dataframe:DataFrame)

# end

In [151]:
n_train_chol = length(y_train_chol)

sex = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.Sex[i] == "M")
        sex[i] = 1;
    end
end
train_cholesterol[!, :Sex] = sex;
first(train_cholesterol, 5)

,ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG
,Int64,Int64,Int64,String,Int64,Int64,Int64,String
1,1,69,1,ASY,140,110,1,Normal
2,2,60,1,ASY,140,293,0,LVH
3,4,46,1,NAP,120,230,0,Normal
4,6,46,1,NAP,150,231,0,Normal
5,7,45,0,ATA,112,160,0,Normal


In [152]:
ChestPainType1 = zeros(Int, n_train_chol)
ChestPainType2 = zeros(Int, n_train_chol)
ChestPainType3 = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.ChestPainType[i] == "ATA")
        ChestPainType1[i] = 1;
    elseif (train_cholesterol.ChestPainType[i] == "NAP")
        ChestPainType2[i] = 1; 
    elseif (train_cholesterol.ChestPainType[i] == "ASY")
        ChestPainType3[i] = 1;
    end
end
select!(train_cholesterol, Not(:ChestPainType))
train_cholesterol[!, :ChestPainType1] = ChestPainType1;
train_cholesterol[!, :ChestPainType2] = ChestPainType2;
train_cholesterol[!, :ChestPainType3] = ChestPainType3;

In [153]:
restingECG1 = zeros(Int, n_train_chol)
restingECG2 = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.RestingECG[i] == "ST")
        restingECG1[i] = 1;
    elseif (train_cholesterol.RestingECG[i] == "LVH")
        restingECG2[i] = 1;  
    end
end
select!(train_cholesterol, Not(:RestingECG))
train_cholesterol[!, :RestingECG1] = restingECG1;
train_cholesterol[!, :RestingECG2] = restingECG2;
first(train_cholesterol, 3)

,ID,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String,Float64
1,1,69,1,140,110,1,109,Y,1.5
2,2,60,1,140,293,0,170,N,1.2
3,4,46,1,120,230,0,150,N,0.0


In [154]:
exerciseAngina = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.ExerciseAngina[i] == "Y")
        exerciseAngina[i] = 1;
    end
end
train_cholesterol[!, :ExerciseAngina] = exerciseAngina;

In [155]:
exerciseAngina = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.ExerciseAngina[i] == "Y")
        exerciseAngina[i] = 1;
    end
end
train_cholesterol[!, :ExerciseAngina] = exerciseAngina;

In [156]:
STSlope1 = zeros(Int, n_train_chol)
STSlope2 = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.STSlope[i] == "Flat")
        STSlope1[i] = 1;
    elseif (train_cholesterol.STSlope[i] == "Down")
        STSlope2[i] = 1;  
    end
end
select!(data, Not(:STSlope))
train_cholesterol[!, :STSlope1] = STSlope1;
train_cholesterol[!, :STSlope2] = STSlope2;



In [157]:
STSlope1 = zeros(Int, n_train_chol)
STSlope2 = zeros(Int, n_train_chol)

for i in 1:n_train_chol
    if (train_cholesterol.STSlope[i] == "Flat")
        STSlope1[i] = 1;
    elseif (train_cholesterol.STSlope[i] == "Down")
        STSlope2[i] = 1;  
    end
end
select!(train_cholesterol, Not(:STSlope))
train_cholesterol[!, :STSlope1] = STSlope1;
train_cholesterol[!, :STSlope2] = STSlope2;

In [158]:
variables = propertynames(select(train_cholesterol, Not([:ID, :HeartDisease])))

15-element Array{Symbol,1}:
 :Age
 :Sex
 :RestingBP
 :Cholesterol
 :FastingBS
 :MaxHR
 :ExerciseAngina
 :Oldpeak
 :ChestPainType1
 :ChestPainType2
 :ChestPainType3
 :RestingECG1
 :RestingECG2
 :STSlope1
 :STSlope2

In [202]:
# Variable d'intérêt
y_train_chol = train_cholesterol.Cholesterol

# Taille d'échantillon
n_train_chol = length(y_train_chol)

# Matrice de structure
# Ne pas prendre Heart Disease
X = hcat(ones(n_train_chol), Matrix(train_cholesterol[:,2:end-1]))

565×16 Array{Float64,2}:
 1.0  69.0  1.0  140.0  110.0  1.0  …  1.0  0.0  0.0  1.0  0.0  0.0  1.0
 1.0  60.0  1.0  140.0  293.0  0.0     1.0  0.0  0.0  1.0  0.0  1.0  1.0
 1.0  46.0  1.0  120.0  230.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  46.0  1.0  150.0  231.0  0.0     1.0  0.0  1.0  0.0  0.0  0.0  1.0
 1.0  45.0  0.0  112.0  160.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  1.0
 1.0  47.0  1.0  140.0  276.0  1.0  …  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  43.0  1.0  150.0  247.0  0.0     1.0  0.0  0.0  1.0  0.0  0.0  1.0
 1.0  49.0  0.0  130.0  269.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  43.0  1.0  130.0  315.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  36.0  1.0  150.0  160.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  50.0  1.0  145.0  264.0  0.0  …  1.0  0.0  0.0  1.0  0.0  0.0  1.0
 1.0  37.0  1.0  118.0  240.0  0.0     0.0  0.0  1.0  0.0  0.0  1.0  1.0
 1.0  41.0  0.0  125.0  184.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        

In [203]:
ȳ = mean(y_train_chol)
SST = sum( (y_train_chol .- ȳ).^2 )

2.096587886725664e6

In [204]:
# Estimation des coefficients de régression
β̂ = X\y_train_chol

# Projection des points sur l'hyperplan de régression
ŷ = X*β̂

# Calcul des erreurs observées
e = y_train_chol - ŷ

# Calcul de la variabilité résiduelle (erreur)
SSE = sum( e.^2 )

# Calcul de la variabilité expliquée par la régression
SSR = SST - SSE

2.096587886725664e6

In [205]:
println(SSE)
println(SST)
R2 = (SSR/SST)

6.0019062024111285e-24
2.096587886725664e6


1.0

In [206]:
ŷ = X*β̂

565-element Array{Float64,1}:
 110.0000000000001
 293.0000000000001
 230.00000000000009
 231.00000000000009
 160.00000000000006
 276.0000000000001
 247.00000000000009
 269.0000000000001
 315.0000000000001
 160.00000000000006
 264.0000000000001
 240.00000000000009
 184.00000000000006
   ⋮
 307.0000000000001
 227.0000000000001
 171.00000000000006
 185.00000000000009
 190.00000000000006
 177.00000000000006
 230.00000000000009
 220.00000000000009
 196.00000000000009
 223.0000000000001
 161.00000000000006
 211.00000000000009